In [1]:
import urllib
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd


url_wikipedia = 'https://ru.wikipedia.org'
url_es = 'https://ru.wikipedia.org/wiki/Категория:Города_по_субъектам_Российской_Федерации'
url_es_q  = urllib.parse.quote(url_es, safe='/:')
print(url_es_q)

https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8


In [7]:
es_bs = BeautifulSoup(urllib.request.urlopen(url_es_q), 'lxml')

In [8]:
all_es = []
# for i in range(30):
for links in es_bs.find_all('div', class_='mw-category-group'):
    for l in links.find_all('a'):
        all_es.append((l['title'], url_wikipedia+l['href']))

In [9]:
len(all_es)

87

In [10]:

all_es_prop = []

for i, (n, lnk) in enumerate(all_es[:30]):
    es = BeautifulSoup(urllib.request.urlopen(lnk), 'lxml')
    es_pr = {}
    city_info = {}
    
    all_cities_in_group = []
    group_cities_links = []
    saved_th = []
    cities = []

    
    mw_pages = es.find("div", {"id": "mw-pages"})
    if mw_pages is not None:
        group_cities = mw_pages.find_all("a")
        for each_a in group_cities:
            if "Городские" in each_a["title"]:
                group_cities_links.append(f"https://ru.wikipedia.org{each_a['href']}")
                all_cities_in_group.append(each_a["title"])
                
    for link in group_cities_links:
        cs = BeautifulSoup(urllib.request.urlopen(link), 'lxml')
        cities_table = cs.find("table", class_="standard")
        if cities_table is not None:
            for row in cities_table.find_all("tr"):
                th = row.find_all("th")
                if th:
                    for i in range(len(th)):
                        saved_th.append(th[i].text.strip())
                td = row.find_all('td')
                if td:
                    for i in range(len(td)):
                        es_pr[saved_th[i]] = td[i].text.strip()
                        if i == len(td)- 1:
                            es_pr["Область"] = n.split(":")[1]
                            cities.append(es_pr)
                            es_pr = {}

    if len(cities) > 0:
        all_es_prop.append({"Область": n.split(":")[1], "Города": cities})
        

In [11]:
pd.json_normalize(all_es_prop[0]["Города"]).set_index("Область")

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб
Область,,,,,,,
Города Крыма,1,Алупка,Ялта,↗9063[4],960,1938,
Города Крыма,2,Алушта,Алушта,↗31 364[4],VI в.,1902,
Города Крыма,3,Армянск,Армянск,↘20 692[4],1736,1993,
Города Крыма,4,Бахчисарай,Бахчисарайский район,↗28 609[4],1389,,
Города Крыма,5,Белогорск,Белогорский район,↗17 445[4],XIII в.,,
Города Крыма,6,Джанкой,Джанкой,↘37 014[4],1855,1917,
Города Крыма,7,Евпатория,Евпатория,↘107 877[4],497 до н. э.,,
Города Крыма,8,Керчь,Керчь,↗154 621[4],VII в. до н. э.,,
Города Крыма,9,Красноперекопск,Красноперекопск,↗25 569[4],1932,1966,


In [12]:
pd.json_normalize(all_es_prop[6]["Города"])

,№,название,район / ЗАТО / город областного значения[4],население(чел.),основание / первое упоминание,статус города,герб,Область
0,1,Астрахань,Астрахань,↘475 629[3],1558,1717,,Города Астраханской области
1,2,Ахтубинск,Ахтубинский район,↘35 635[3],1793,1959,,Города Астраханской области
2,3,Знаменск,"Знаменск, ЗАТО",↘24 628[3],1948,1962,,Города Астраханской области
3,4,Камызяк,Камызякский район,↗16 154[3],1560,1973,,Города Астраханской области
4,5,Нариманов,Наримановский район,↗11 104[3],1963,1984,,Города Астраханской области
5,6,Харабали,Харабалинский район,↗18 514[3],1789,1974,,Города Астраханской области


In [13]:
print(len(all_es_prop))

28


In [14]:

all_df = pd.json_normalize(all_es_prop[0]["Города"])


for i in range(len(all_es_prop) - 1):
    cities = pd.json_normalize(all_es_prop[i+1]["Города"])
    all_df = pd.concat([all_df, cities], names=["123"])
    
all_df

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб,Область,город республиканского значения (городской округ),основан,...,площадь (км²),район / город областного значения[6],городской/муниципальный округ[7],район[4] / город республиканского значения[5],основание /первое упоминание,муниципальный район / городской округ,район/город[3],мун.район / городской округ[4],район / город республиканского значения[6],мун. район / городской округ[7]
0,1,Алупка,Ялта,↗9063[4],960,1938,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Алушта,Алушта,↗31 364[4],VI в.,1902,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Армянск,Армянск,↘20 692[4],1736,1993,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Бахчисарай,Бахчисарайский район,↗28 609[4],1389,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Белогорск,Белогорский район,↗17 445[4],XIII в.,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,Питкяранта(карел. Pitkyrandu),NaN,↘9818[4],NaN,1940,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1499,NaN,NaN,NaN,Питкярантский район,Питкярантский мун.район
9,10,Пудож(карел. Puudoži),NaN,↘8387[4],NaN,1785,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1382,NaN,NaN,NaN,Пудожский район,Пудожский мун.район
10,11,Сегежа(карел. Segeža),NaN,↘25 334[4],NaN,1943,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1914,NaN,NaN,NaN,Сегежский район,Сегежский мун.район
11,12,Сортавала(карел. Sortavala),NaN,↘18 600[4],NaN,1783,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1468,NaN,NaN,NaN,Сортавала,Сортавальский мун.район


In [15]:
#Сколько городов в каждой области
all_df.groupby("Область")["название"].count()

Область
Города Адыгеи                           2
Города Алтайского края                 12
Города Амурской области                10
Города Архангельской области            0
Города Астраханской области             6
Города Башкортостана                    0
Города Белгородской области            11
Города Брянской области                16
Города Бурятии                          6
Города Владимирской области            23
Города Волгоградской области            0
Города Вологодской области             15
Города Воронежской области             15
Города Дагестана                       10
Города Еврейской автономной области     0
Города Забайкальского края              0
Города Ивановской области              17
Города Ингушетии                        5
Города Иркутской области                0
Города Кабардино-Балкарии               0
Города Калининградской области         22
Города Калмыкии                         3
Города Калужской области               22
Города Камчатского края   

In [16]:
all_df.count()

№                                                  329
название                                           232
район[5] / город   республиканского значения[6]     16
население(чел.)                                    240
основание / первое упоминание                      191
                                                  ... 
муниципальный район  / городской округ              22
район/город[3]                                       3
мун.район  / городской округ[4]                      3
район / город  республиканского значения[6]         13
мун. район /  городской округ[7]                    13
Length: 69, dtype: int64

In [17]:
all_df.dtypes

№                                                  object
название                                           object
район[5] / город   республиканского значения[6]    object
население(чел.)                                    object
основание / первое упоминание                      object
                                                    ...  
муниципальный район  / городской округ             object
район/город[3]                                     object
мун.район  / городской округ[4]                    object
район / город  республиканского значения[6]        object
мун. район /  городской округ[7]                   object
Length: 69, dtype: object

In [18]:
import re

all_df["население(чел.)"] = all_df["население(чел.)"].astype(str).apply(lambda x: "".join(re.findall('[0-9]+', x)[:-1]))

In [19]:
all_df["население(чел.)"] = pd.to_numeric(all_df["население(чел.)"], downcast="integer")

In [20]:
all_df

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб,Область,город республиканского значения (городской округ),основан,...,площадь (км²),район / город областного значения[6],городской/муниципальный округ[7],район[4] / город республиканского значения[5],основание /первое упоминание,муниципальный район / городской округ,район/город[3],мун.район / городской округ[4],район / город республиканского значения[6],мун. район / городской округ[7]
0,1,Алупка,Ялта,9063.0,960,1938,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Алушта,Алушта,31364.0,VI в.,1902,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Армянск,Армянск,20692.0,1736,1993,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Бахчисарай,Бахчисарайский район,28609.0,1389,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Белогорск,Белогорский район,17445.0,XIII в.,,,Города Крыма,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,Питкяранта(карел. Pitkyrandu),NaN,9818.0,NaN,1940,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1499,NaN,NaN,NaN,Питкярантский район,Питкярантский мун.район
9,10,Пудож(карел. Puudoži),NaN,8387.0,NaN,1785,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1382,NaN,NaN,NaN,Пудожский район,Пудожский мун.район
10,11,Сегежа(карел. Segeža),NaN,25334.0,NaN,1943,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1914,NaN,NaN,NaN,Сегежский район,Сегежский мун.район
11,12,Сортавала(карел. Sortavala),NaN,18600.0,NaN,1783,,Города Республики Карелия,NaN,NaN,...,NaN,NaN,NaN,NaN,1468,NaN,NaN,NaN,Сортавала,Сортавальский мун.район


In [21]:
all_df.groupby("Область")["население(чел.)"].describe()

,count,mean,std,min,25%,50%,75%,max
Область,,,,,,,,
Города Адыгеи,2.0,78280.000000,92072.373978,13175.0,45727.50,78280.0,110832.50,143385.0
Города Алтайского края,12.0,103476.666667,175824.227535,10187.0,17236.25,34525.5,90830.75,630877.0
Города Амурской области,10.0,45558.400000,71162.268589,7057.0,11160.50,17951.0,43544.00,241437.0
Города Архангельской области,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Города Астраханской области,6.0,96944.000000,185708.115688,11104.0,16744.00,21571.0,32883.25,475629.0
Города Башкортостана,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Города Белгородской области,11.0,74293.636364,107842.562612,5629.0,12736.50,33032.0,62452.50,339978.0
Города Брянской области,16.0,41822.250000,91070.276278,5270.0,12042.50,16829.0,23237.50,379152.0
Города Бурятии,6.0,86643.166667,172089.406247,4368.0,12993.00,21055.0,24396.50,437565.0


In [67]:
all_df[all_df["Область"].str.startswith("Города А") == True]["название"]

0          Адыгейск
1            Майкоп
0            Алейск
1           Барнаул
2        Белокуриха
3             Бийск
4            Горняк
5           Заринск
6       Змеиногорск
7     Камень-на-Оби
8       Новоалтайск
9          Рубцовск
10        Славгород
11           Яровое
0         Белогорск
1      Благовещенск
2         Завитинск
3               Зея
4        Райчихинск
5         Свободный
6       Сковородино
7             Тында
8       Циолковский
9         Шимановск
0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7               NaN
8               NaN
9               NaN
10              NaN
11              NaN
12              NaN
0         Астрахань
1         Ахтубинск
2          Знаменск
3           Камызяк
4         Нариманов
5          Харабали
Name: название, dtype: object

In [65]:
all_df["Область"]

0                  Города Крыма
1                  Города Крыма
2                  Города Крыма
3                  Города Крыма
4                  Города Крыма
                ...            
8     Города Республики Карелия
9     Города Республики Карелия
10    Города Республики Карелия
11    Города Республики Карелия
12    Города Республики Карелия
Name: Область, Length: 329, dtype: object